In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
def save_dataset(dataset, path, sep=';', index=False, encoding='utf-8-sig'):
    dataset.to_csv(path, sep=sep, index=index, encoding=encoding)

def read_dataset(path, sep=';', encoding='utf-8-sig', low_memory=False):
    return pd.read_csv(path, sep=sep, encoding=encoding, low_memory=low_memory)

df = read_dataset('../../data/datasets/dataset_1_3.csv')

In [3]:
df["Дата получения оценки"] = pd.to_datetime(df["Дата получения оценки"], errors="coerce")

df_1_1 = df[
    (df["Сквозной семестр"] < 3) &
    (df["Год начала обучения"] == 2023) &
    (df["Дата получения оценки"] < "2024-09-01")
].copy()

C:\Users\333\AppData\Local\Temp\ipykernel_25328\1376014555.py:1: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Дата получения оценки"] = pd.to_datetime(df["Дата получения оценки"], errors="coerce")


In [4]:
df_1_2 = df_1_1[df_1_1['Оценка'] != 'уваж.']
df_1_2 = df_1_2[df_1_2["Оценка"] != "невбрн."]

In [5]:
is_na = {'зач.': 70, 'неуваж.': 0, 'недсд.': 0, '4': 70, 'недоп.': 0, '5': 90, 'незач.': 20, '2': 20, '3': 50}

df_1_2["Балл"] = df_1_2["Балл"].fillna(df_1_2["Оценка"].map(is_na))

In [6]:
retries_count = df_1_2.groupby(["UUID студента", "Наименование дисциплины"]).size().reset_index(name="Попытки")
df_1_3 = df_1_2.loc[
    df_1_2.groupby(["UUID студента", "Наименование дисциплины"])["Балл"].idxmax()
].reset_index(drop=True)
df_1_3 = df_1_3.merge(retries_count, on=["UUID студента", "Наименование дисциплины"])
df_1_3["Количество пересдач"] = df_1_3["Попытки"] - 1
df_1_3.drop(columns=["Попытки"], inplace=True)

In [7]:
total_retries = df_1_3.groupby("UUID студента")["Количество пересдач"].sum().reset_index(
    name="Общее количество пересдач"
)

df_1_3 = df_1_3.merge(total_retries, on="UUID студента", how="left")

In [8]:
df_1_4 = df_1_3.drop(columns=["Количество пересдач", "Наименование дисциплины + семестр"]).copy()

In [9]:
mask = (df["Сквозной семестр"] > 2) & (df["Состояние студента"] != "Активный")
students_to_activate = df.loc[mask, "UUID студента"].unique()

df_1_4.loc[df_1_4["UUID студента"].isin(students_to_activate), "Состояние студента"] = "Активный"

In [10]:
debt_grades = ['неуваж.', 'недсд.', 'недоп.', 'незач.', '2']

df_1_5 = df_1_4.copy()

df_1_5['is_debt'] = df_1_5['Оценка'].isin(debt_grades)

total_debts_per_student = (
    df_1_5.groupby("UUID студента")['is_debt']
    .sum()
    .reset_index(name='Общее количество долгов')
)

total_debts_per_student['Общее количество долгов'] = total_debts_per_student[
    'Общее количество долгов'
].astype(int)

df_1_5.drop(columns=['is_debt'], inplace=True)

df_1_5 = df_1_5.merge(total_debts_per_student, on="UUID студента", how='left')

if 'Количество долгов' in df_1_5.columns:
    df_1_5.drop(columns=['Количество долгов'], inplace=True)

In [11]:
df_1_6 = df_1_5.copy()

In [12]:
def power_penalty_score(student_scores, subject_stats, p=2.0):
    total_score = 0
    subject_count = 0

    for subject, student_score in student_scores.items():
        stats = subject_stats[subject]
        scores = stats['scores']
        total_students = stats['total_students']

        non_zero_scores = [s for s in scores if s > 0]
        num_fails = total_students - len(non_zero_scores)

        if len(non_zero_scores) == 0 or total_students == 0:
            continue

        mean_clean = sum(non_zero_scores) / len(non_zero_scores)
        fail_ratio = num_fails / total_students

        if student_score < 40:
            if 1 / (fail_ratio + 1e-6) <= 0:
                print(1 / (fail_ratio + 1e-6))
                print(f'fail_ratio: {fail_ratio}')
                print(f'num_fails: {num_fails}')
                print(f'non_zero_scores: {len(non_zero_scores)}')
                print(f'total_students: {total_students}')
                print(f"subject: {subject}")
            multiplier = 1 + math.log(1 / (fail_ratio + 1e-6))
            adjusted = (mean_clean ** p) * multiplier
        else:
            delta = mean_clean - student_score
            signed_power_delta = math.copysign(abs(delta) ** p, delta)
            adjusted = signed_power_delta * fail_ratio

        total_score += adjusted
        subject_count += 1

    if subject_count == 0:
        return None

    return total_score / subject_count

In [13]:
def make_subject_stats(df):
    ans = {}
    for index, row in df.iterrows():
        subj = row["Наименование дисциплины"]
        student = row["UUID студента"]
        score = row["Балл"]
        oc = row["Оценка"]
        
        if subj not in ans:
            ans[subj] = {'scores': [], 'total_students': 0}
        if pd.isna(score):
            score = is_na[oc][0]
        ans[subj]['scores'].append(score)
        ans[subj]['total_students'] += 1
    return ans

In [14]:
def process_scores(df):
    ans = {}
    subject_stats = make_subject_stats(df)

    current_student = None
    student_data = {}

    for index, row in df.iterrows():
        student = row["UUID студента"]
        subject = row["Наименование дисциплины"]
        score = row["Балл"]
        oc = row["Оценка"]

        if student != current_student:
            if current_student is not None:
                ans[current_student] = power_penalty_score(student_data, subject_stats)
            current_student = student
            student_data = {}
        if pd.isna(score):
            score = is_na[oc][0]
        student_data[subject] = score
    
    if current_student is not None:
        ans[current_student] = power_penalty_score(student_data, subject_stats)
    return ans

In [15]:
scores = process_scores(df_1_6)

In [16]:
df_1_6["Рейтинг студента"] = df_1_6["UUID студента"].map(scores)

In [17]:
missing_uuids = df_1_6[df_1_6["Рейтинг студента"].isna()]["UUID студента"].unique()

if len(missing_uuids) > 0:
    print("Нет рейтинга для следующих UUID:")
    print(missing_uuids)
else:
    print("Все UUID успешно получили рейтинг.")

Все UUID успешно получили рейтинг.


In [18]:
def check_nuniques(df):
    nuniques = df.groupby("UUID студента").nunique()
    for column in nuniques.columns.to_list():
        print(f"{column} - {nuniques[column].nunique()} - {nuniques[column].unique()}")

In [19]:
df_1_7 = df_1_6.copy()

In [20]:
df_1_7['Балл за инд. достижения'] = df_1_7['Балл за инд. достижения'].fillna(0)
df_1_7["Балл за инд. достижения"] = df_1_7["Балл за инд. достижения"].astype(int)

In [21]:
benefit = df_1_7["Льготы"].unique()
df_1_7.loc[df_1_7["Категория конкурса БВИ"].isin(benefit), "Категория конкурса БВИ"] = np.nan
df_1_7['Категория конкурса БВИ'] = np.where(df_1_7['Категория конкурса БВИ'].notna(), '1', "0")
print(df_1_7['Категория конкурса БВИ'].unique())
df_1_7['Категория конкурса БВИ'] = df_1_7['Категория конкурса БВИ'].astype(int)

['0' '1']


In [22]:
columns_to_drop = ["Наименование олимпиады", "Контрольное мероприятие", "Тип оценки", "Наименование дисциплины", "Модель реализации",
                   "Оценка", "Балл", "UUID оценки", "Дата получения оценки", "Семестр","Год", "Наименование дисциплины ИТС", 
                   "ИТС наименование в дипломе", "Год начала обучения", "Курс", "Активное состояние", "Место рождения"]

df_1_7 = df_1_7.drop(columns=columns_to_drop)

In [23]:
df_1_8 = df_1_7.drop_duplicates(subset=['UUID студента'], keep='first').copy()

In [24]:
df_1_8.shape

(2281, 52)

In [25]:
df_1_8["Код направления 1"] = np.nan
df_1_8["Код направления 3"] = np.nan


df_1_8[["Код направления 1", "Код направления 3"]] = df_1_8["Код направления подготовки"].apply(
    lambda x: pd.Series({
        "Код направления 1": int(x.split(".")[0]),
        "Код направления 3": int(x.split(".")[2])
    })
)

In [26]:
df_1_8 = df_1_8.drop(columns=["Код направления подготовки", "Наименование напр. подготовки", "Образовательная программа", "Сквозной семестр", "ЛН студента", "ЛН студента (архив)"])

In [27]:
col_to_drop = ["Формирующее подразделение", "Область образования", "Целевик", "Хоз. группа | Вкл. обучение", 
               "Малоимущий", "Слушатель", "В архиве", "Приказ об отчислении", "Акад. отп. по сост. здор.", "Акад. отп. по уходу за реб.", 
               "UUID персоны", "Адрес регистрации. Страна", "Страна законченного уч. зав.",
               "Название законченного уч. зав.", "Регион законченного уч. зав.", "Квалификация зак.уч.зав.", "Специальность зак.уч.зав.", "Документы подтв. льготы"]

df_1_8 = df_1_8.drop(columns=col_to_drop)

In [28]:
dummies = pd.get_dummies(df_1_8["Основа освоения (сокр.)"], drop_first=True)
df_1_8["Основа освоения (сокр.)"] = dummies  # "бюджет"=0, "контракт"=1
df_1_8 = df_1_8.rename(columns={"Основа освоения (сокр.)": "Контракт"})

In [29]:
df_1_8["Контракт"] = df_1_8["Контракт"].astype(int)

In [30]:
df_1_8 = df_1_8.drop(columns=["Срок освоения"])

In [31]:
dummies = pd.get_dummies(df_1_8["Нуждается в общежитии"])
dummies = dummies.drop("Нет", axis=1)
df_1_8["Нуждается в общежитии"] = dummies
df_1_8["Нуждается в общежитии"] = df_1_8["Нуждается в общежитии"].astype(int)

In [32]:
dummies = pd.get_dummies(df_1_8["Иностранный абитуриент (МОН)"])
dummies = dummies.drop("Нет", axis=1)
df_1_8["Иностранный абитуриент (МОН)"] = dummies
df_1_8["Иностранный абитуриент (МОН)"] = df_1_8["Иностранный абитуриент (МОН)"].astype(int)

In [33]:
dummies = pd.get_dummies(df_1_8["Пол"])
dummies = dummies.drop("Ж", axis=1)
df_1_8["Пол"] = dummies
df_1_8["Пол"] = df_1_8["Пол"].astype(int)

In [34]:
citizenship_map = {
    "Russia": ["Российская Федерация"],
    "PostSoviet": [
        "Республика Азербайджан", "Республика Армения", "Республика Беларусь", "Грузия", "Республика Казахстан", "Киргизская Республика",
        "Латвия", "Литва", "Республика Молдова", "Республика Таджикистан", "Туркменистан", "Республика Узбекистан", "Украина", "Эстония"
    ]
}

citizenship_class = []
citizenship = df_1_8["Гражданство"].tolist()

for current in citizenship:
    if current in citizenship_map.get("Russia"):
        citizenship_class.append("Russia")
    elif current in citizenship_map.get("PostSoviet"):
        citizenship_class.append("PostSoviet")
    else:
        citizenship_class.append("others")

citizenship_series = pd.Series(citizenship_class)
df_1_8["citizenship_class"] = citizenship_series

In [35]:
df_1_8["fromEkaterinburg"] = df_1_8["Адрес регистрации. Город"].apply(
    lambda x: 1 if x == "Екатеринбург" else 0
)

In [36]:
df_1_8["fromSverdlovskRegion"] = df_1_8["Адрес регистрации. Область"].apply(
    lambda x: 1 if x == "Свердловская область" else 0
)

In [37]:
df_1_8 = df_1_8.drop(columns=["Город законченного уч. зав.", "Адрес регистрации. Область", "Адрес регистрации. Город"])

In [38]:
current_df = df_1_8[
    (df_1_8["Тип законченного уч. зав."] == "Высшее") & 
    (df_1_8["Тип документа об образовании"].isin(["Аттестат о среднем общем образовании", "Диплом о среднем профессиональном образовании"]))
]
uuid_to_replace = current_df["UUID студента"]
for uuid in uuid_to_replace:
    df_1_8.loc[df_1_8["UUID студента"] == uuid, "Тип законченного уч. зав."] = "Техникум"

current_df = df_1_8[
    (df_1_8["Тип законченного уч. зав."] == "Высшее") & 
    (df_1_8["Тип документа об образовании"].isin(["Академическая справка"]))
]
uuid_to_replace = current_df["UUID студента"]
for uuid in uuid_to_replace:
    df_1_8.loc[df_1_8["UUID студента"] == uuid, "Тип законченного уч. зав."] = "Техникум"

current_df = df_1_8[
    df_1_8["Тип законченного уч. зав."].isin(["(не указано)", "Вечерняя школа", "Основная общеобразовательная школа", "Средняя общеобразовательная школа"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Тип законченного уч. зав."] = "Школа"

current_df = df_1_8[
    df_1_8["Тип законченного уч. зав."].isin(["Гимназия", "Лицей", "Дневная школа, лицей, гимназия"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Тип законченного уч. зав."] = "Профильная Школа"

current_df = df_1_8[
    df_1_8["Тип законченного уч. зав."].isin(["Кадетский корпус", "Суворовское училище"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Тип законченного уч. зав."] = "Военное уч. заведение"

current_df = df_1_8[
    df_1_8["Тип законченного уч. зав."].isin(["Колледж", "Техникум", "Профессиональное училище"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Тип законченного уч. зав."] = "СПО"

In [39]:
df_1_8 = df_1_8.drop("Тип документа об образовании", axis=1)

In [40]:
year_of_end = df_1_8["Год окончание уч. зав."]
df_1_8["Год окончание уч. зав."] = 2023 - year_of_end
df_1_8 = df_1_8.rename(columns={"Год окончание уч. зав.": "Прошло лет с окончания уч. заведения"})

In [41]:
df_1_8["Льготы"] = df_1_8["Льготы"].fillna("Отсутствует")

current_df = df_1_8[
    df_1_8["Льготы"].isin(["02 Лица из числа детей-сирот (до 23 лет)", "03 Дети, оставшиеся без попечения родителей (до 18 лет)", "05 Лица из числа детей без попечения родителей (до 23 лет)"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Льготы"] = "Сироты"

current_df = df_1_8[
    df_1_8["Льготы"].isin(["04 Дети-инвалиды", "07 Инвалиды с детства", "11 Инвалиды II группы"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Льготы"] = "Инвалиды"

current_df = df_1_8[
    df_1_8["Льготы"].isin([
        "10 Ветераны боевых действий",
        "17 Дети военнослужащих, сотрудников федеральных органов исполнительной власти и федеральных государственных органов, направленных в другие государства",
        "19 Дети лиц, принимавших участие в специальной военной операции на территориях Украины",
        "21 Дети военнослужащих, погибших или получивших увечье при исполнении обязанностей военной службы в специальной военной операции на территориях Украины"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Льготы"] = "Боевые действия"

current_df = df_1_8[
    df_1_8["Льготы"].isin([
        "09 Лица, поступающие по направлению Министерства в пределах квоты, установленной Правительством Российской Федерации"]
)]
uuid_to_replace = current_df["UUID студента"]
df_1_8.loc[df_1_8["UUID студента"].isin(uuid_to_replace), "Льготы"] = "Квота для иностранных граждан"

In [42]:
df_1_8["Тип олимпиады"] = df_1_8["Тип олимпиады"].fillna("Отсутствует")

In [43]:
HDI_dict = {
    'Арабская Республика Египет': 0.731,
    'Габонская Республика': 0.706,
    'Йеменская Республика': 0.455,
    'Киргизская Республика': 0.692,
    'Китайская Народная Республика': 0.768,
    'Королевство Марокко': 0.683,
    'Народная Республика Бангладеш': 0.661,
    'Республика Азербайджан': 0.745,
    'Республика Армения': 0.759,
    'Республика Беларусь': 0.808,
    'Республика Гаити': 0.535,
    'Республика Индонезия': 0.705,
    'Республика Ирак': 0.686,
    'Республика Казахстан': 0.811,
    'Республика Камерун': 0.576,
    'Республика Колумбия': 0.752,
    "Республика Кот д'Ивуар": 0.550,
    'Республика Куба': 0.764,
    'Республика Молдова': 0.767,
    'Республика Перу': 0.762,
    'Республика Судан': 0.508,
    'Республика Таджикистан': 0.685,
    'Республика Узбекистан': 0.727,
    'Российская Федерация': 0.822,
    'Сирийская Арабская Республика': 0.577,
    'Социалистическая Республика Вьетнам': 0.703,
    'Турецкая Республика': 0.838,
    'Туркменистан': 0.745,
    'Украина': 0.773,
    'Федеративная Демократическая Республика Эфиопия': 0.498,
    'Федеративная Республика Бразилия': 0.754,
    'Федеративная Республика Германия': 0.942,
    'Федеративная Республика Нигерия': 0.535
}

In [44]:
df_1_8['Human Development Index'] = df_1_8['Гражданство'].map(HDI_dict)

In [45]:
date_of_birth = df_1_8["Дата рождения"]
date_of_birth = pd.to_datetime(date_of_birth, format='%d.%m.%Y')

start_styding_date = pd.to_datetime("1.08.2023", format='%d.%m.%Y')
start_styding_column = (start_styding_date - date_of_birth)
start_styding_column = start_styding_column.dt.days // 365

df_1_8["Дата рождения"] = start_styding_column
df_1_8 = df_1_8.rename(columns={"Дата рождения": "Полных лет на момент поступления"})
df_1_8["Полных лет на момент поступления"] = df_1_8["Полных лет на момент поступления"].astype(int)

In [46]:
df_1_8 = df_1_8.drop(columns=["UUID студента", "Гражданство"])

In [47]:
dummies = pd.get_dummies(df_1_8["Вид конкурса"])
dummies = dummies.drop("Основные места", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Тип олимпиады"])
dummies = dummies.drop("Отсутствует", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Форма освоения"])
dummies = dummies.drop("Очная", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Уровень подготовки"])
dummies = dummies.drop("Бакалавр", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Тип законченного уч. зав."])
dummies = dummies.drop("Школа", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Льготы"])
dummies = dummies.drop("Отсутствует", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["citizenship_class"])
dummies = dummies.drop("Russia", axis=1)
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

In [48]:
dummies = pd.get_dummies(df_1_8["Код направления 1"])
dummies = dummies.drop(9, axis=1)
dummies = dummies.rename(columns={10: "Код направления 1: 10"})
dummies = dummies.rename(columns={11: "Код направления 1: 11"})
dummies = dummies.rename(columns={27: "Код направления 1: 27"})
dummies = dummies.rename(columns={29: "Код направления 1: 29"})
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

dummies = pd.get_dummies(df_1_8["Код направления 3"])
dummies = dummies.drop(1, axis=1)
dummies = dummies.rename(columns={2: "Код направления 3: 2"})
dummies = dummies.rename(columns={3: "Код направления 3: 3"})
dummies = dummies.rename(columns={4: "Код направления 3: 4"})
dummies = dummies.astype(int)
df_1_8 = df_1_8.join(dummies, how='left')

In [49]:
df_1_8 = df_1_8.drop(columns=["Тип олимпиады", "Вид конкурса", "Форма освоения", "Уровень подготовки", "Тип законченного уч. зав.", "Льготы", "citizenship_class", "Код направления 1", "Код направления 3"])

In [50]:
mask = df_1_8["Состояние студента"].isin(["Отп.акад.", "Отп.б.посещ.", "Отчислен"])
df_1_8.loc[mask, "Состояние студента"] = "1"

mask = df_1_8["Состояние студента"].isin(["Активный", "Зак. с дипломом"])
df_1_8.loc[mask, "Состояние студента"] = "0"

df_1_8["Состояние студента"].unique()

array(['0', '1'], dtype=object)

In [51]:
df_1_8["Состояние студента"] = df_1_8["Состояние студента"].astype(int)
df_1_8 = df_1_8.rename(columns={"Состояние студента": "Таргет"})

In [52]:
# Магистры
df_magistr = df_1_8[df_1_8["Магистр"] == 1].copy()
df_magistr.drop(columns=["Магистр", "Специалист"], inplace=True)

# Специалисты и бакалавры
df_spec_bak = df_1_8[
    (df_1_8["Специалист"] == 1) | ((df_1_8["Специалист"] == 0) & (df_1_8["Магистр"] == 0))
].copy()
df_spec_bak.drop(columns=["Магистр"], inplace=True)

save_dataset(df_magistr, "../../data/datasets/subjects_magistr_v2.csv")
save_dataset(df_spec_bak, '../../data/datasets/subjects_bak_spec_v2.csv')